# Psi4 Examples
This notebook accompaies the Medium article `Doing Quantum Chemistry through Python`. The examples below are written in June 2023 using Psi4 version 1.8. 

Copyright (c) 2023 James L. McDonagh

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [ ]:
from typing import Union, Tuple
import psi4
import os
import numpy as np
import logging

logging.basicConfig(format="%(message)s")
log = logging.getLogger(__name__)
log.setLevel("INFO")

In [ ]:
psi4.__version__

## Global options

Psi4 has a large list of global options which we can set for our calculations. The next cell provides a list of the global options. We can then set some of these global options with a dictionary of `{$OPTION: $VALUE}`. your can reset all options to the initial set using `psi4.core.clean_options()`.

In [ ]:
psi4.core.print_global_options()

In [ ]:
psi4.set_options(
    {
        "SAVE_OPTIMIZATION": True,
        "MAXITER": 100,
        "GEOM_MAXITER": 100,
        "FULL_HESS_EVERY": 5,
        "PRINT": 2,
        "GUESS": "sad",
        "REFERENCE": "uhf",
        "SCF_TYPE": "direct",
    }
)

We can also set the path for psi4 to save tempory files. This is particularly useful if you want to keep certain tempory files for restart purposes e.g.
```
psi4_io.set_specific_path(32, './')
psi4_io.set_specific_retention(32, True)
```

To keep the restart unit 32 in the current directory.

In this case so we can see everyting we set the path to the current working directory for all files. In general this isn't the best plan but for learning it can be helpful.

In [ ]:
psi4_io = psi4.core.IOManager.shared_object()

In [ ]:
psi4_io.set_default_path(os.getcwd())

## Energy and Geometry Optimization Calculations
This section covers some of the most common qunatum chemistry calculations through the Psi4 python API. Other examples can be found on Psi4's own [GitHub](https://github.com/psi4/psi4/tree/master/samples/python) pages.

### Energy calculations
Lets try a few examples to calculate the energy of water with different methods. In the next section we set the mmeory avaliable to use for the calculations and then an output file. If we don't set an output file it will output to the screen.

In [ ]:
psi4.set_memory("1GB")
psi4.set_output_file("p4_output.txt", append=False, loglevel=20, print_header=True, inherit_loglevel=True)
psi4.core.set_num_threads(4)

In [ ]:
molecule = psi4.geometry(
    """
    0 1
    O     0.00000     0.64670    -0.01863
    H     0.76026     0.61622    -0.62453
    H    -0.76026     0.61622    -0.62453
    units angstrom
    symmetry c1
    """ 
)

In [ ]:
energy = psi4.energy("hf/cc-pvdz")
log.info("The energy for this configuration is {:.6f} Hartree".format(energy))

We can clean up all of the tempory files with one command

In [ ]:
psi4.core.clean()

### Geometry Optimization

In this section we can use the molecule as defined for the energy calculation to optimize its geometry

In [ ]:
molecule = psi4.geometry(
    """
    0 1
    O     0.00000     0.64670    -0.01863
    H     0.76026     0.61622    -0.62453
    H    -0.76026     0.61622    -0.62453
    units angstrom
    symmetry c1
    """ 
)

In [ ]:
opt_energy = psi4.optimize("hf/cc-pvdz")
log.info("The optimized energy for this configuration is {:.6f} Hartree".format(opt_energy))

In [ ]:
log.info("Difference in energy from initial conformation to optimized conformation {:.6f} Hartree".format(energy - opt_energy))

In [ ]:
psi4.core.clean()

This is great for a small example but what about for larger examples that might not converge in the number steps? We can use a pythonic `try` and `except` statement and return the wavefunction object to allow us to restart. Lets simulate this with our water example we will give only two steps to optimze and catach the expection then continue with more steps.

In [ ]:
molecule = psi4.geometry(
    """
    0 1
    O     0.10000     0.74670    -0.02863
    H     0.76026     0.61622    -0.62453
    H    -0.76026     0.61622    -0.62453
    units angstrom
    symmetry c1
    """ 
)

In [ ]:
psi4.set_options(
    {
        "MAXITER": 2,
        "GEOM_MAXITER": 2
    }
)

In [ ]:
try:
    opt_energy = psi4.optimize("hf/cc-pvdz")
    log.info("The optimized energy for this configuration is {:.6f} Hartree".format(opt_energy))
    unconv = False
except psi4.driver.ConvergenceError as cerr:
    log.warning("Geometry unconverged will try restarting")
    unconv_wfn = cerr.wfn
    unconv_wfn.to_file(unconv_wfn.get_scratch_filename(180))
    psi4.set_options(
        {
            "GUESS": "read",
            "MAXITER": 100,
            "GEOM_MAXITER": 100,
        }
    )
    psi4.optimize("hf/cc-pvdz")
    log.info("The optimized energy for this configuration is {:.6f} Hartree".format(opt_energy))


We can construct this in to a general function for optimization

In [ ]:
def psi4_optimize(initial_iterations: int = 2,
                  increment_iteration: int = 2,
                  max_loop_count: int = 20,
                  molecule: Union[psi4.core.Molecule, None] = None,
                  method: str = "hf/cc-pvdz"
                 ) -> Tuple[float, psi4.core.Wavefunction, dict]:
    """
    General function to run Psi4 optimization and restart a max of max_loop_count times incrementing the number
    of iterations by increment_iteration from initial_iterations each loop
    :param initial_iterations: integer the number of initial iterations
    :param increment_iteration: integer the number of iteration increment by each time it fails to converge
    :param max_loop_count: integer the maximum number of times to increment for convergence incomplete
    :param molecule: psi4.core.Molecule the molecule object or None if given will save last geometry of each unconverged loop
    :method: string define the quantum chem optimization method eg hf/cc-pvdz
    :return: Tuple[float, psi4.core.Wavefunction, list] the energy, the wavefunction object and the history of conformations
    """
    
    log = logging.getLogger(__name__)
    
    unconv = True

    psi4.set_options(
        {
            "MAXITER": initial_iterations,
            "GEOM_MAXITER": initial_iterations
        }
    )
    
    loop_count = 0
    
    if molecule is not None:
        molecule_traj = []

    while unconv is True:
        
        try:
            
            log.info("\ncount {}\n".format(loop_count))
            opt_energy, wfn, traj = psi4.optimize(method, return_wfn=True, return_history=True)
            log.info("The optimized energy for this configuration is {:.6f} Hartree".format(opt_energy))
            unconv = False
            
        except psi4.driver.ConvergenceError as cerr:
            
            log.warning("Geometry unconverged will try restarting")
            if molecule is not None:
                bohr_coor = molecule.geometry()
                bohr_coor.scale(psi4.constants.bohr2angstroms)
                molecule_traj.append(bohr_coor.to_array())
            unconv_wfn = cerr.wfn
            unconv_wfn.to_file(unconv_wfn.get_scratch_filename(180))
            psi4.set_options(
                {
                    "GUESS": "read",
                    "MAXITER": initial_iterations + increment_iteration,
                    "GEOM_MAXITER": initial_iterations + increment_iteration
                }
            )
            
            loop_count = loop_count + 1

            if max_loop_count <= loop_count:
                log.error("Unconverged in maximum number of loops")
                raise cerr
                
    if molecule is not None:
        traj = {"coordinates": tuple(np.array(molecule_traj))}

    return opt_energy, wfn, traj

In [ ]:
molecule = psi4.geometry(
    """
    0 1
    O     0.00000     0.74670    -0.01863
    H     0.76026     0.61622    -0.62453
    H    -0.76026     0.61622    -0.62453
    units angstrom
    symmetry c1
    """ 
)

psi4.set_memory("1GB")
psi4.set_output_file("p4_output.txt", append=False, loglevel=20, print_header=True, inherit_loglevel=True)
psi4.core.set_num_threads(4)

#Include optional molecule=molecule to get a trajectory of the final geometry of each loop inplace of the psi4 history
energy, wfn, traj = psi4_optimize(initial_iterations=10,
                                  increment_iteration=10,
                                  max_loop_count=20,
                                  method="hf/cc-pvdz",
                                 )

log.info("Optimized energy {}".format(energy))

psi4.core.clean()

## One electron properties

Having reached a minimum Psi4 allows you calculate one electron proerties

In [ ]:
oeprops = psi4.core.OEProp(wfn)
oeprops.add("DIPOLE")
oeprops.add("QUADRUPOLE")
oeprops.add("MULLIKEN_CHARGES")
oeprops.add("MULTIPOLE(4)")
oeprops.add("ESP_AT_NUCLEI")
oeprops.add("MO_EXTENTS")
oeprops.add("LOWDIN_CHARGES")
oeprops.add("WIBERG_LOWDIN_INDICES")
oeprops.add("MAYER_INDICES")
oeprops.add("NO_OCCUPATIONS")
oeprops.compute()

In [ ]:
properties = wfn.variables()
properties["DIPOLE"]

## Interaction energy calculation

In this section we show how to calculate binding energy with a counterpoise correction to the energy.

In [ ]:
molecule = psi4.geometry(
    """
    0 1
    O        0.00000        0.64670       -0.01863
    H        0.76026        0.61622       -0.62453
    H       -0.76026        0.61622       -0.62453
    --
    0 1
    O        0.00000       -0.04191        2.64300
    H        0.00000        0.08820        1.66859
    H        0.00000        0.87457        2.95608
    units angstrom
    symmetry c1
    """ 
)

In [ ]:
interaction_energy = psi4.energy("hf/cc-pvdz", bsse_type="cp")
log.info("Interaction energy {} Hartree".format(interaction_energy))

In [ ]:
psi4.core.clean()